In [1]:
#Utilizacion de Word2Vec 

In [3]:
import bs4 as bs
import urllib.request
import re
import nltk
import pandas as pd
pd.set_option('max_colwidth', -1)
import string
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df = pd.read_csv('../dataset/train.csv')

In [23]:
#Al igual que en otros notebooks, preprocesamiento previo
# Quitamos las urls
df['text'] = df['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['text'] = df['text'].apply(clean_text)

df['word_count'] = df['text'].apply(lambda x: len(x.split(' ')))

In [24]:
df.head()

,id,keyword,location,text,target,word_count
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1,7
1,4,NaN,NaN,forest fire near la ronge sask canada,1,7
2,5,NaN,NaN,residents asked shelter place notified officers evacuation shelter place orders expected,1,11
3,6,NaN,NaN,13000 people receive wildfires evacuation orders california,1,8
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pours school,1,10


In [49]:
#Para que esto funcione, se requiere que todas las palabras de todos los textos de los tweets esten en una lista
palabras = []
def extraer_palabras(string):
    splitted = string.split(' ')
    for palabra in splitted:
        palabras.append(palabra)
    
df['text'].apply(extraer_palabras)
from gensim.models import Word2Vec


word2vec = Word2Vec(palabras, min_count=2)
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'d': <gensim.models.keyedvectors.Vocab object at 0x000001CDB8A171D0>, 'e': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08ACF8>, 's': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AD68>, 'r': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08ADD8>, 'a': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AE10>, 'o': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AE48>, 'n': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AE80>, 't': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AEB8>, 'h': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AEF0>, 'q': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AF28>, 'u': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AF60>, 'k': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AF98>, 'm': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC08AFD0>, 'y': <gensim.models.keyedvectors.Vocab object at 0x000001CDBC094048>, 'l': <gensim.models

In [32]:
v1 = word2vec.wv['m']